## 3.2 Implémentation des relations 1:n

### Exemple sous SQLAlchemy

In [1]:
from sqlalchemy import Column,  ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker,  relationship

# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

<img src="../Relation_1_n.png">

<p id="declarations" style="font-size:120%">Déclaration des classes métier :</p>

In [2]:
# Classe de base
Base = declarative_base()
Base.metadata.bind = engine

# Définition des classes
class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    pseudo = Column(String(16), nullable=False)
    email = Column(String(255), nullable=False)
    pwd = Column(String(32), nullable=False)
    
    office_id = Column(Integer,ForeignKey("office.id"))
    office = relationship("Office", uselist=False, back_populates="user")
    
    address_id = Column(Integer,ForeignKey("address.id"))
    address = relationship("Address", uselist=False)

class Office(Base):
    __tablename__ = 'office'
    
    id = Column(Integer, primary_key=True)
    building = Column(String(32),nullable=False)
    floor = Column(Integer,nullable=False)
    room_id = Column(String(32),nullable=False)
    
    user=relationship("User", uselist=False, back_populates="office")
    
class Address(Base):
    __tablename__ = 'address'
    
    id = Column(Integer, primary_key=True)
    detail = Column(String(255),nullable=False)
    street_name = Column(String(255),nullable=False)
    number = Column(Integer,nullable=False)
    city = Column(String(255),nullable=False)
    zipcode = Column(Integer,nullable=False)

<p id="usage" style="font-size:120%">Usage :</p>

In [3]:
# Session
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Récupération d'un utilisateur
user = session.query(User).filter(User.pseudo == 'rde').one()
print(user.pseudo, user.address)

rde <__main__.Address object at 0x000002AA48054080>


In [4]:
# Création d'une adresse
address = session.query(Address).filter(Address.detail=="Résidence Paul Comparat").first()
if address == None:
    address = Address(
        detail="Résidence Paul Comparat",
        street_name="Chemin des Mouilles",
        number=51,
        city="Ecully",
        zipcode="69130")

# Affectation de cette adresse à l'utilisateur
user.address = address

session.add(user)
session.commit()

In [8]:
users = session.query(User).\
    join(Address).\
    filter(Address.detail == 'Résidence Paul Comparat').\
    all()
    
for user in users:
    print(user.pseudo, user.email, user.address.detail)

rde Raymond.Deubaze@gmail.com Résidence Paul Comparat
jbt jean.bonnot@ec-lyon.fr Résidence Paul Comparat
